# Thalamic nuclei mean kurtosis estimation

> Import ROIs

In [13]:
import pandas as pd
import os
import re
import numpy as np
import nibabel as nb
import argparse
import textwrap

> Set input subjects / file locations

In [4]:
dataLoc = os.getcwd()
subjects = [x for x in os.listdir(dataLoc) if x.startswith('N') or x.startswith('F')]

In [34]:
def get_segment_map(subject,side):
    '''
    Returns nibabel matrix of 
    the connectivity-based segmentation map of thalamus
    '''
    segment_file = os.path.join(dataLoc,
                              subject,
                              'segmentation',
                              side,
                              'dki_biggest.nii.gz')
    f = nb.load(segment_file)
    segment_map = f.get_data()
    return segment_map

In [35]:
def get_mk_map(subject):
    '''
    Returns nibabel matrix of
    the whole brain mean kurtosis image
    '''
    mk_file = os.path.join(dataLoc,
                         subject,
                         'DKI',
                         'kmean.nii')
    f = nb.load(mk_file)
    mk_map = f.get_data()
    return mk_map

In [59]:
def get_mean_MK_for_nuclei(segment_map, mk_map, cortex):
    '''
    Returns mean of the MK, and the volume 
    of the thalamic segment
    in connection to sepcific cortex
    '''
    nuclei_dict = {"LPFC":1,
                  "LTC":2,
                  "MPFC":3,
                  "MTC":4,
                  "OCC":5,
                  "OFC":6,
                  "PC":7,
                  "SMC":8}
    nucleiNum = nuclei_dict[cortex]
    cortexMap = segment_map == nucleiNum
    segment_volume = np.sum(cortexMap)
    
    segment_mk = cortexMap * mk_map
    mk_mean = segment_mk[segment_mk!=0].mean()

    return mk_mean, segment_volume

In [68]:
def get_connectivity_for_nuclei(subject, side, cortex):
    '''
    Returns the sum of connectivity
    in the thalamic segment
    '''
    if side == 'left':
        side_s = 'lh'
    else:
        side_s = 'rh'
    
    connectivity_file = os.path.join(dataLoc, 
                                    subject,
                                    'segmentation',
                                    side,'seeds_to_{side_s}_{cortex}.nii.gz'.format(side_s=side_s,
                                                                                   cortex=cortex))
    f = nb.load(connectivity_file)
    connectivit_map = f.get_data()
    
    totalConnectivity = connectivit_map.sum()
    return totalConnectivity
    

In [76]:
def thalamic_mk(subject, side):
    '''
    Returns a pandas dataframe,
    containing MK of thalamic segments 
    for a given subject
    '''
    group = subject[:3]
    segment_map = get_segment_map(subject, side)
    mk_map = get_mk_map(subject)
    
    df = pd.DataFrame()
    for cortex in ["LPFC", "LTC", "MPFC", "MTC", "OCC", "OFC", "PC", "SMC"]:
        totalConnectivity = get_connectivity_for_nuclei(subject, side, cortex)
        mk_mean, segment_volume = get_mean_MK_for_nuclei(segment_map, mk_map, cortex)
        cortex_df = pd.DataFrame(data = [[mk_mean, segment_volume,totalConnectivity,cortex, side, group]], 
                                 index=[subject], 
                                 columns=['MK','nucleiVolume','totalConnectivity','cortex','side','group'])
        df = pd.concat([df, cortex_df])
    
    df = df['relativeConnectivity'] = df.totalConnectivity / df.totalConnectivity.sum()
    return df

In [ ]:
thalamic_mk('NOR26_YJH','left')

In [75]:
df

,MK,nucleiVolume,totalConnectivity,cortex,side,group,relativeConnectivity
NOR26_YJH,0.830991446972,137,2467689,LPFC,left,NOR,0.135294
NOR26_YJH,0.722406148911,128,3109955,LTC,left,NOR,0.170507
NOR26_YJH,0.741360068321,71,2214287,MPFC,left,NOR,0.121401
NOR26_YJH,0.724031865597,6,1907565,MTC,left,NOR,0.104585
NOR26_YJH,nan,0,321940,OCC,left,NOR,0.017651
NOR26_YJH,1.07476615906,10,641725,OFC,left,NOR,0.035183
NOR26_YJH,0.856482565403,138,2327863,PC,left,NOR,0.127628
NOR26_YJH,0.787861406803,126,5248410,SMC,left,NOR,0.287751


In [64]:
thalamic_mk('NOR81_KSA','left')

,MK,volume,cortex,side,group
NOR81_KSA,0.973071753979,117,LPFC,left,NOR
NOR81_KSA,0.670243799686,1,LTC,left,NOR
NOR81_KSA,1.10995388031,6,MPFC,left,NOR
NOR81_KSA,0.633870244026,151,MTC,left,NOR
NOR81_KSA,nan,0,OCC,left,NOR
NOR81_KSA,1.1396958828,80,OFC,left,NOR
NOR81_KSA,0.956947147846,209,PC,left,NOR
NOR81_KSA,0.766120910645,30,SMC,left,NOR


In [65]:
thalamic_mk('NOR60_KSH','left')

,MK,volume,cortex,side,group
NOR60_KSH,0.815050959587,51,LPFC,left,NOR
NOR60_KSH,0.82516336441,174,LTC,left,NOR
NOR60_KSH,1.06589615345,204,MPFC,left,NOR
NOR60_KSH,nan,0,MTC,left,NOR
NOR60_KSH,0.820995926857,9,OCC,left,NOR
NOR60_KSH,0.628565847874,89,OFC,left,NOR
NOR60_KSH,1.0065125227,114,PC,left,NOR
NOR60_KSH,1.13429534435,157,SMC,left,NOR


In [42]:
def main():
    '''
    Estimates the mean kurtosis in the thalamic segments
    for all subjects.
    The thalamic segments is obtained from the connectivity-based segmentation
    from FSL.
    
    It returns mk_data.csv containing all subjects' data.
    
    '''
    allData = pd.DataFrame()
    for subject in subjects:
        for side in ['left','right']:
            subjectDf = thalamic_mk(subject, side)
            allData = pd.concat([allData, subjectDf])
    
    allData.to_csv('mk_data.csv')

```
if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter,
        description=textwrap.dedent('''\
            {codeName} : Estimates mean kurtosis in the thalamic nuclei
            ===========================================================
            '''.format(codeName=os.path.basename(__file__))))

    parser.add_argument(
        '-s', '--subjects',
        help='subject list',
        nargs='+')

    parser.add_argument(
        '-side', '--side',
        help='side')
    
    args = parser.parse_args()    
    thalamic_mk(args)
```